In [ ]:
import pandas as pd

In [ ]:
dataset_train1 = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv', index_col='id')
dataset_test1 = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col='id')
y = dataset_train1.claim
y = pd.DataFrame(y)
dataset_train = dataset_train1.drop(['claim'], axis=1)

ratio

In [ ]:
dataset_train['f10_ratio_f9'] = dataset_train['f10']/dataset_train['f9']
dataset_train['f92_ratio_f73'] = dataset_train['f92']/dataset_train['f73']
dataset_train['f78_ratio_f73'] = dataset_train['f78']/dataset_train['f73']
dataset_train['f89_ratio_f73'] = dataset_train['f89']/dataset_train['f73']
dataset_train['f90_ratio_f74'] = dataset_train['f90']/dataset_train['f74']
dataset_train['f43_ratio_f74'] = dataset_train['f43']/dataset_train['f74']
dataset_train['f36_ratio_f73'] = dataset_train['f36']/dataset_train['f73']
dataset_train['f32_ratio_f35'] = dataset_train['f32']/dataset_train['f35']
dataset_train['f5_ratio_f86'] = dataset_train['f5']/dataset_train['f86']
dataset_train['f14_ratio_f26'] = dataset_train['f14']/dataset_train['f26']
dataset_train['f63_ratio_f26'] = dataset_train['f63']/dataset_train['f26']

In [ ]:
dataset_test1['f10_ratio_f9'] = dataset_test1['f10']/dataset_test1['f9']
dataset_test1['f92_ratio_f73'] = dataset_test1['f92']/dataset_test1['f73']
dataset_test1['f78_ratio_f73'] = dataset_test1['f78']/dataset_test1['f73']
dataset_test1['f89_ratio_f73'] = dataset_test1['f89']/dataset_test1['f73']
dataset_test1['f90_ratio_f74'] = dataset_test1['f90']/dataset_test1['f74']
dataset_test1['f43_ratio_f74'] = dataset_test1['f43']/dataset_test1['f74']
dataset_test1['f36_ratio_f73'] = dataset_test1['f36']/dataset_test1['f73']
dataset_test1['f32_ratio_f35'] = dataset_test1['f32']/dataset_test1['f35']
dataset_test1['f5_ratio_f86'] = dataset_test1['f5']/dataset_test1['f86']
dataset_test1['f14_ratio_f26'] = dataset_test1['f14']/dataset_test1['f26']
dataset_test1['f63_ratio_f26'] = dataset_test1['f63']/dataset_test1['f26']

nan count

In [ ]:
dataset_train['nan_count'] = dataset_train.isnull().sum(axis=1)
dataset_train['nan_count']  = dataset_train['nan_count']/dataset_train['nan_count'].max()

In [ ]:
dataset_test1['nan_count'] = dataset_test1.isnull().sum(axis=1)
dataset_test1['nan_count']  = dataset_test1['nan_count']/dataset_test1['nan_count'].max()

not nan count

In [ ]:
dataset_train['count'] = dataset_train.notnull().sum(axis=1)
dataset_train['count']  = dataset_train['count']/dataset_train['count'].max()

In [ ]:
dataset_test1['count'] = dataset_test1.notnull().sum(axis=1)
dataset_test1['count']  = dataset_test1['count']/dataset_test1['count'].max()

sum all cols

In [ ]:
dataset_train['sum'] = dataset_train.sum(axis=1)
dataset_train['sum'].head()

In [ ]:
dataset_test1['sum'] = dataset_test1.sum(axis=1)
dataset_test1['sum'].head()

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, PowerTransformer

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
dataset_train = imputer.fit_transform(dataset_train)
dataset_test = imputer.transform(dataset_test1)

In [ ]:
#pt = PowerTransformer()
#dataset_train = pt.fit_transform(dataset_train)
#dataset_test = pt.transform(dataset_test)

In [ ]:
qt = QuantileTransformer(n_quantiles=256, output_distribution='uniform')
dataset_train = qt.fit_transform(dataset_train)
dataset_test = qt.transform(dataset_test)

In [ ]:
#kb = KBinsDiscretizer(n_bins=1024, encode='ordinal',strategy='uniform')
#dataset_train = kb.fit_transform(dataset_train)
#dataset_test = kb.transform(dataset_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(dataset_train)
dataset_test_sc = x_scaler.transform(dataset_test)

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
def model_builder(lr):
    """Модель нейронной сети для агента"""
    inputA = keras.Input(shape=(132))
    line = Reshape((132,1))(inputA)
    #line = Conv1D(filters=16, kernel_size=1, activation='relu')(line)
    #line = BatchNormalization()(line)
    line = Conv1D(filters=32, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.2)(line)
    
    line = Conv1D(filters=64, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    line = Flatten()(line)
    line = Dense(64, activation='relu')(line)
    line = Dropout(0.5)(line)
    #line = Dense(64, activation='relu')(line)
    outputA = Dense(units=1, activation="sigmoid", kernel_regularizer=keras.regularizers.l1(0.01))(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=lr), metrics=[tf.keras.metrics.AUC(name='auc'), 'binary_accuracy'],)
    return model

In [ ]:
lr=0.0005
with strategy.scope():
    model = model_builder(lr)

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_auc',
    mode='max',
    verbose=1,
    save_best_only=True)

In [ ]:
#model = load_model('/models/best.h5')

In [ ]:
EPOCHS = 500
#EPOCHS = 1
model.fit(dataset_train_sc, y, validation_split=0.2, epochs=EPOCHS, callbacks=[save_model_callback,], batch_size=4096)

In [ ]:
model = load_model('best.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
output = pd.DataFrame({'Id': dataset_test1.index,'claim': preds[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 